In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle

from network_wrangler import Scenario
from lasso.seflorida.seflorida_roadway import SEFloridaRoadwayNetwork
from lasso.seflorida.seflorida_parameters import SEFloridaParameters

In [ ]:
print("setup basic paths...")
root_dir = "projects/serpm/"
input_dir = os.path.join(root_dir, "data", "interim", "8-standard-format")
external_dir = os.path.join(root_dir, "data", "external")
output_dir = os.path.join(root_dir, "data", "processed")
card_dir = os.path.join(root_dir, "data", "project_cards")

print("read parameters...")
lasso_dir = os.path.join(root_dir, "Lasso")
parameters = SEFloridaParameters(lasso_base_dir=lasso_dir)

In [ ]:
print("read pickle...")
version_01_pickle_file_name = f"{output_dir}/working_scenario_01.pickle"
v_01_scenario = pickle.load(open(version_01_pickle_file_name, "rb"))

## Create Scenario and Apply Project Cards

In [ ]:
# create scenario
print("create scenario...")
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df[
    "shstGeometryId"
]

step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory=card_dir,
    tags=["year 2019"],
    validate_project_cards=False,
)

In [ ]:
step1_scenario.apply_all_projects()

In [ ]:
step1_scenario.applied_projects

## Make Travel Model Networks

In [ ]:
# Make Travel Model Networks
print("create travel model network...")
model_net = SEFloridaRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object=step1_scenario.road_net, parameters=parameters
)

print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

In [ ]:
## add centroids & connectors
model_net = model_net.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

In [ ]:
# add pnr nodes & connectors
max_node_id = model_net.nodes_df["model_node_id"].max()
serpm8_nodes = gpd.read_file(f"{external_dir}/4-third-party/serpm8/serpm8_nodes.shp")
pnr_nodes = serpm8_nodes[serpm8_nodes["ACTIVEFLAG"]==1].sort_values("STATIONNUMB").to_crs(4326).reset_index(drop=True)
pnr_nodes = pnr_nodes[["X", "Y", "geometry"]]
pnr_nodes["model_node_id"] = max_node_id + pnr_nodes.index + 1
pnr_nodes["osm_node_id"] = ''
pnr_nodes["shst_node_id"] = ''
pnr_nodes["drive_access"] = 1
pnr_nodes["walk_access"] = 1
pnr_nodes["bike_access"] = 1
pnr_nodes["rail_only"] = 0
pnr_nodes["farezone"] = 0
pnr_nodes["pnr"] = 1
pnr_nodes["X"] = pnr_nodes["geometry"].x
pnr_nodes["Y"] = pnr_nodes["geometry"].y

model_net = model_net.build_pnr_connections(
    roadway_network = model_net,
    pnr_nodes = pnr_nodes,
    parameters = parameters,
    build_taz_walk_connector = True)

In [ ]:
print("convert standard roadway to seflorida network...")
model_net = model_net.roadway_standard_to_seflorida_network(model_net, parameters, output_proj=2236)

In [ ]:
model_net.links_seflorida_df = model_net.links_df.copy()
model_net.nodes_seflorida_df = model_net.nodes_df.copy()

## Export results

In [ ]:
print("write as fixedwidth...")
model_net.write_roadway_as_fixedwidth(
    output_link_txt=os.path.join(output_dir, "links.txt"),
    output_node_txt=os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt=os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt=os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script=os.path.join(
        output_dir, "make_complete_network_from_fixed_width_file.s"
    ),
)

In [ ]:
model_net.write_roadway_as_shp(
    output_link_shp=os.path.join(output_dir, "links.shp"),
    output_node_shp=os.path.join(output_dir, "nodes.shp"),
    data_to_csv=False,
    data_to_dbf=True,
)

#### save pickle file

In [ ]:
v2_scenario = {"road_net": model_net, "transit_net": v_01_scenario.transit_net}
working_scenario = Scenario.create_scenario(base_scenario = v2_scenario)

working_scenario_filename = os.path.join(output_dir, 'working_scenario_02.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))